Le notebook Rentabilité contient le coeur de notre projet : en indiquant une position de départ, une quantité de carburant et le type de carburant, notre fonction va retourner la station essence dans laquelle il est le plus rentable d'aller faire le plein. On prend en compte la distance, la consommation, le prix du trajet et le prix du carburant dans la station d'arrivée 

In [76]:
import pandas as pd
import module_projet as md
from geopy.geocoders import Nominatim



On récupère les données, on travaille ici sur la base de données instantanées

In [ ]:
dat_inst = pd.read_csv('data/fichier_inst.csv', sep= ';')

Nous avons besoin d'informations sur l'utilisateur pour faire marcher notre algorithme

In [ ]:
adresse=input("renseigner votre adresse")
carburant=input("Renseigner le carburant souhaité")
quanti=input("renseigner le nombre de litre souhaité")

df_renta=md.data_carb(dat_inst, carburant, ['id', 'ville', 'geom','prix_valeur','prix_nom'])

In [ ]:

#fonctions a récuperer dans le md

def Adresse_to_gps(Adresse) : 

    geolocator = Nominatim(user_agent="arman.akgonul@ensae.fr")
    location = geolocator.geocode(Adresse, timeout=10000)
    return str(str(location.latitude) + ','+ str(location.longitude))


def data_carb(df, carb, colonnes_utiles):
    data_1 = df[df.prix_nom == carb]
    d = data_1[colonnes_utiles]
    d.drop_duplicates(keep = 'first', inplace=True) 
    return d

# a ajouter : un traitement des NaN
    return d


On a donc besoin d'une fonction qui calcule le prix du plein en fonction de la station d'essence choisie

In [80]:
df_test=df_renta.sample(50) #à remplacer par le df contentenant les plus proches stations
df_opti=df_test.assign(cout=1) #nouvelle colonne avec le cout du plein en tenant compte du trajet et du prix



def prix_plein(Quantité, position, station) : 
    A = list(map(float,position.split(',')))
    B = list(map(float,df_opti.iloc[station,2].split(',')))
    
    D= distance1(A,B)/100    #calcul de la distance avec la station 
    conso_trajet=D*0.06 #calcul du nb de litre d'essence necessaire pour aller a la station
    P=Quantité+conso_trajet #calcul du nb de litre necessaire pour faire le plein après le trajet
    prix_L=df_opti.iloc[station,3] #prix au litre à la station
    px_plein=prix_L*P #prix du plein de base + prix du trajet
    return px_plein



Ici on fait une boucle qui calcule le prix du plein pour toutes les stations pour trouver celle où il est le plus rentable d'aller

In [ ]:

position=Adresse_to_gps(adresse) #on sort cette ligne de la boucle, sinon le package "Nomatim" nous bloque pour usage trop récurrent
for i in range(len(df_opti)) : 
    C=prix_plein(50,position,i) #on calcule le prix du plein pour chaque station
    df_opti.iloc[i,5]=C #on ajoute ce prix dans le dataframe

df_opti = df_opti.sort_values('cout', ascending=True)


Enfin, on affiche la meilleure station calculée par l'algorithme

In [85]:
print(df_opti.head(1)) #station la plus rentable

            id   ville          geom  prix_valeur prix_nom   cout
17572  8190001  ASFELD  49.465,4.112        1.697   Gazole  84.85
